In [1]:
%pip install pandas praw python-dotenv openpyxl pysentimiento 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw
import pandas as pd
from dotenv import load_dotenv
import os
import datetime
import concurrent.futures
from pysentimiento import create_analyzer

In [3]:
# Carga las variables de entorno del archivo .env
load_dotenv()

# Accede a las variables de entorno
client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')


In [4]:
# https://www.reddit.com/prefs/apps
reddit = praw.Reddit(
   client_id= client_id , # personal use script
   client_secret= client_secret ,# secret
   user_agent= user_agent  # my_user_agent
)

In [5]:
def translate_emotion_to_spanish(emotion):
    translations = {
        'joy': 'alegría',
        'surprise': 'sorpresa',
        'sadness': 'tristeza',
        'fear': 'miedo',
        'anger': 'enojo',
        'disgust': 'asco'
    }
    return translations.get(emotion, emotion)

emotion_analyzer  = create_analyzer(task="emotion", lang="es")

def analyze_sentiment(text):
    result = emotion_analyzer.predict(text)
    if result.output == 'others':
        del result.probas['others']
        max_emotion = max(result.probas, key=result.probas.get)
        
        return translate_emotion_to_spanish(max_emotion)
    else:
        return translate_emotion_to_spanish(result.output)


def create_data_dict(comment):
    if comment.body and len(comment.body.split()) <= 128:
        return {
            'Texto': comment.body,
            'Fecha': datetime.datetime.fromtimestamp(comment.created).strftime('%d-%m-%Y'),
            'Nombre Usuario': comment.author.name if comment.author else '',
            'Concepto asociado': analyze_sentiment(comment.body),
            'Fuente': f'https://www.reddit.com{comment.permalink}',
            'Género': "N/A",
            'Grupo Étareo': "N/A",
            'Escolaridad': "N/A"
        }

In [6]:
n_comments = 1
data = []

# Descarga los posts más populares del subreddit 'chile'
hot_posts = reddit.subreddit('chile').hot(limit=n_comments)

# Extrae los comentarios del post
for post in hot_posts:
    post.comments.replace_more(limit=n_comments)
    comments = post.comments.list()
    
    # Analiza el sentimiento de cada comentario y guarda los resultados
    with concurrent.futures.ThreadPoolExecutor() as executor:
        data.extend(filter(None, executor.map(create_data_dict, comments)))  # Filtra los comentarios None antes de agregarlos a la lista.

# Crea un DataFrame con los datos
df = pd.DataFrame(data)

In [7]:
# Guarda el dataframe en un archivo CSV
# df.to_csv('reddit_data.csv', index=False)

# Guarda el dataframe en un archivo Excel
df.to_excel('reddit_data.xlsx', index=False)